<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 15px; height: 80px">


# Project 4 
## Web Scraping for Indeed.com and Predicting Salaries

## Stage 2: Data cleaning

In [1]:
import pandas as pd
import re

### Load data

In [3]:
jobs = pd.read_csv('../GA/DSI15-project-submissions/Julia-McAleenan/project-4/jobs.csv', index_col=0)
jobs.shape

(18137, 7)

In [5]:
jobs.head(2)

,company,location,job_title,salary,job_summary,company_rating,link
0,Deutsche Bank,London,Anti-Fraud Bribery and Corruption Data Scientist,NaN,"Demonstrated experience in platforms (SAS, R e...",3.8,/rc/clk?jk=5025a182b2e067ac&fccid=f1d8e147024a...
1,UK Government - Institute for Apprenticeships ...,London SW1W,Data Scientist,"£29,522 - £36,222 a year",Development of new data visualisations.\nWorki...,NaN,/rc/clk?jk=ad71b08ead9c768a&fccid=e863441157f2...


### Data cleaning - initial steps
- Remove rows with no salary information
- Remove duplicated rows

In [6]:
# remove rows with no salary information
df = jobs[jobs.salary.notnull()].copy()
df.shape

(6572, 7)

In [7]:
# remove duplicate rows and reset index
# exclude 'link' from the check for duplicates as different links 
# may still lead to the same job posting
df.drop_duplicates(subset=['company', 'location', 'job_title', 'salary', 'job_summary',
       'company_rating'], inplace=True)
df.reset_index(drop=True, inplace=True)
df.shape

(1921, 7)

### Data cleaning - salary
The salary information is contained in a string and is sometimes quoted annually, sometimes monthly, sometimes weekly, sometimes daily and sometimes hourly. It is often quoted as a range.
 - Add a new column 'salary_type' to describe how the salary is quoted (annual, daily etc.)
 - Add a new column 'annualised_salary' with the annualised salary for each job listing. Where a range is quoted, assume the mid-point. 

In [12]:
def salary_type(string):
    '''
    function to return type of salary quoted
    (annual, monthly, weekly, daily or hourly)
    '''
    if 'year' in string:
        return 'annual'
    elif 'month' in string:
        return 'monthly'
    elif 'week' in string:
        return 'weekly'
    elif 'day' in string:
        return 'daily'
    elif 'hour' in string:
        return 'hourly'
    else:
        return np.nan

In [13]:
# create new 'salary type' feature (annual, daily etc)
df['salary_type'] = df['salary'].map(salary_type)

In [14]:
df.salary_type.value_counts()

annual     1709
daily       114
hourly       78
weekly       14
monthly       6
Name: salary_type, dtype: int64

In [15]:
def annualise_salary(string):
    '''
    function to calcuate annualised salary,
    if a range is given, return mid-point
    - for weekly rates, assume a year consists of 44 working weeks
    - for daily rates, assume a year consists of 220 working days
    - for hourly rates, assume a 37.5 hour working week
    '''
    nums = [float(x.replace(',','')) for x in re.findall(r'£(\d+,*\d*)', string)]
    avg = sum(nums) / len(nums)
    if 'year' in string:
        return avg
    elif 'month' in string:
        return avg * 12
    elif 'week' in string:
        return avg * 44
    elif 'day' in string:
        return avg * 220
    elif 'hour' in string:
        return avg * 37.5 * 44
    else:
        return np.nan

In [16]:
# create new annualised salary feature
df['annualised_salary'] = df['salary'].map(annualise_salary)

### Data cleaning - location
- Some locations contain the postcode as well as the city e.g. SW1W - remove the postcode.
- Drop rows where location is 'England' or 'United Kingdom'
- There are many different towns and cities in the locations column which will not help with the modelling. 
 - Create a new feature 'county' to group the locations together by county. To do this, use a CSV with a list of UK towns and associated counties and also the London Borough Profiles from data.london.gov.uk (https://data.london.gov.uk/dataset/london-borough-profiles).
 - Create another new feature 'region' to group the counties by region (e.g. South East).

In [18]:
# function to remove postcode from the location
def remove_postcode(string):
    string = re.sub(r'([A-Z]+\d+[A-Z]*)', '', string)
    string = re.sub(r'(\d[A-Z][A-Z])', '', string)
    return string.strip()

In [19]:
df['location'] = df['location'].map(remove_postcode)

In [20]:
# drop rows with location = England
df = df[df.location != 'England']

In [21]:
# drop rows with location = United Kingdom
df = df[df.location != 'United Kingdom']

In [25]:
# load CSV with list of UK towns and associated counties
counties = pd.read_csv('../GA/DSI15-project-submissions/Julia-McAleenan/project-4/towns_to_counties.csv')

# create list of counties
counties_list = list(counties.County.unique())

In [29]:
# load CSV containing list of London boroughs
london_boroughs = pd.read_csv('../GA/DSI15-project-submissions/Julia-McAleenan/project-4/london-borough-profiles.csv', encoding='latin')

# create list of 33 London boroughs
london_boroughs_list = list(london_boroughs.Area_name)[:-5]

# add London to the list so it is categorised as 'Greater London'
london_boroughs_list += ['London']

In [30]:
# create dictionary for exceptions not caught by the databases above
# for example 'North London'
counties_dict = {'Greater London': ['East London', 'North London', 'South London', 'West London',
                            'London Bridge Station', 'Blackfriars Station', 'Hammersmith',
                            'South West London', 'East Croydon Station', 'Richmond',
                            'London / Gatwick Airport', 'Middlesex', 'West Drayton',
                            'South Kensington', 'Covent Garden', 'Canary Wharf', 'Hayes',
                            'Heathrow', 'Colindale', 'Kensington', 'Tolworth', 'Feltham',
                            'Tooting', 'Northwood', "King's Cross"],
         'Greater Manchester': ['Cheadle', 'Heald Green'],
         'Surrey': ['Staines-upon-Thames', 'Frimley', 'Addlestone'],
         'Cambridgeshire': ['St. Neots', 'Waterbeach'],
         'Oxfordshire': ['Thames Valley'],
         'Leicestershire': ['Ashby-De-La-Zouch', 'Houghton on the Hill'],
         'Hertfordshire': ['St Albans', 'Apsley', 'Colney Heath'],
         'West Sussex': ['Sussex'],
         'East Sussex': ['Brighton and Hove'],
         'Berkshire': ['Theale'],
         'Buckinghamshire': ['Haddenham', 'Stokenchurch'],
         'Staffordshire': ['Burton-On-Trent'],
         'Cheshire': ['Alderley Edge', 'Cheshire Oaks'],
         'Warwickshire': ['Stratford', 'Stratford-upon-Avon', 'Leamington Spa'],
         'Somerset': ['Weston-super-Mare'],
         'Wiltshire': ['Bradford-on-Avon'],
         'Worcestershire': ['Hindlip'],
         'Merseyside': ['Knowsley'],
         'Gwent': ['Newport'],
         'Mid Glamorgan': ['Nantgarw'],
         'Midlothian': ['Roslin'],
         'Renfrewshire': ['Renfrew'],
         'Lothian': ['Kings Buildings'],
         'South East': ['South East'],
         'South West': ['South West'],
         'Remote': ['Home Based']}

# function to look up in the dictionary above
def find_area(location, areas_dict):
    for area, locations in areas_dict.items():
        if location in locations:
            return area
        else:
            continue
    return np.nan

In [31]:
# function to find the county using different databases
def find_county(location, df=counties, counties=counties_list, london=london_boroughs_list):
    county = df[df.Town == location].County
    if len(county) == 1:
        return county.iloc[0]
    elif location in counties:
        return location
    elif location in london:
        return 'Greater London'
    else:
        return find_area(location, counties_dict)

In [32]:
df['county'] = df['location'].apply(find_county)

In [33]:
# check for null values
df[df.county.isnull()]

,company,location,job_title,salary,job_summary,company_rating,link,salary_type,annualised_salary,county
1244,Verrus,Central Belt,Senior Business Development Manager,"£60,000 - £70,000 a year",Carrying out research and market intelligence ...,NaN,/rc/clk?jk=22a9ad5e0df0969f&fccid=89f3a333c132...,annual,65000.0,NaN
1780,Bright Purple Resourcing,Central Belt,"Data Scientist (""R"")","£35,000 - £50,000 a year",R is being increasingly utilised by the organi...,NaN,/rc/clk?jk=bb6cf6c5ae49128f&fccid=9b585c4a6566...,annual,42500.0,NaN


In [34]:
# remove any remaining null values
df = df[df.county.notnull()]
df.shape

(1914, 10)

In [35]:
df.county.value_counts().head()

Greater London        766
Greater Manchester    142
West Midlands          97
Bristol                83
Berkshire              76
Name: county, dtype: int64

In [39]:
# create further region column
regions = pd.read_csv('../GA/DSI15-project-submissions/Julia-McAleenan/project-4/counties_to_regions.csv')

# create list of regions
regions_list = list(regions.Region.unique())

In [40]:
# create dictionary for Scotland, Wales and any exceptions
regions_dict = {'Remote': ['Remote'],
                'Scotland': ['Strathclyde', 'Lothian', 'Falkirk', 'Fife', 'Midlothian', 'Renfrewshire'],
                'Wales': ['Gwent', 'South Glamorgan', 'Mid Glamorgan',
                          'Wrexham', 'Monmouthshire']}

In [41]:
# function to find the region based on the county
def find_region(county, df=regions, regions=regions_list):
    region = df[df.County == county].Region
    if len(region) == 1:
        return region.iloc[0]
    elif county in regions:
        return county
    else:
        return find_area(county, regions_dict)

In [42]:
df['region'] = df['county'].apply(find_region)

In [44]:
df.region.value_counts()

London                766
South East            262
West Midlands         191
North West            165
South West            110
Yorkshire & Humber    102
East                   80
Scotland               57
Remote                 54
North East             50
East Midlands          50
Wales                  27
Name: region, dtype: int64

In [52]:
# write clean data to CSV
df.to_csv('jobs_clean.csv')